In [51]:
class JumpingHeadTM:
    """
    Máquina de Turing com cabeçote saltador (pula 2 casas por vez).
    - transitions: {(estado, símbolo_lido): (símbolo_escrito, direção, próximo_estado)}
    direção: 'R' = pula 2 p/ direita, 'L' = pula 2 p/ esquerda, 'N' = não move.
    """
    def __init__(self, tape, transitions, start_state, accept_state, reject_state):
        self.tape = list(tape)  # fita como lista mutável
        self.transitions = transitions  # tabela de transições
        self.state = start_state  # estado atual
        self.head = 0  # posição do cabeçote
        self.accept = accept_state  # estado de aceitação
        self.reject = reject_state  # estado de rejeição

    def step(self):
        symbol = self.tape[self.head] if self.head < len(self.tape) else '_'
        key = (self.state, symbol)
        if key in self.transitions:
            new_symbol, direction, next_state = self.transitions[key]
            # expandir fita se necessário
            if self.head >= len(self.tape):
                self.tape.extend(['_'] * (self.head - len(self.tape) + 1))
            self.tape[self.head] = new_symbol
            self.state = next_state
            if direction == 'R':
                self.head += 2
            elif direction == 'L':
                self.head = max(0, self.head - 2)
            # 'N' não move
        else:
            self.state = self.reject

    def run(self, max_steps=1000):
        steps = 0
        while self.state != self.accept and self.state != self.reject and steps < max_steps:
            self.step()
            steps += 1
        return (self.state == self.accept), ''.join(self.tape), steps


Verificar se a cadeia binária contém quantidade par de 1s

In [52]:
t_jump_paridade_1s = {
    # Estado inicial: paridade atual = par
    ('q0', '1'): ('X', 'R', 'q1a'),
    ('q0', '0'): ('0', 'R', 'q0a'),
    ('q0', '_'): ('_', 'N', 'q_acc'),

    # Continuação da paridade par
    ('q0a', '1'): ('X', 'R', 'q1b'),
    ('q0a', '0'): ('0', 'R', 'q0b'),
    ('q0a', '_'): ('_', 'N', 'q_acc'),

    ('q0b', '1'): ('X', 'R', 'q1'),
    ('q0b', '0'): ('0', 'R', 'q0'),
    ('q0b', '_'): ('_', 'N', 'q_acc'),

    # Paridade ímpar (após ler um 1)
    ('q1', '1'): ('X', 'R', 'q0a'),
    ('q1', '0'): ('0', 'R', 'q1a'),
    ('q1', '_'): ('_', 'N', 'q_rej'),

    ('q1a', '1'): ('X', 'R', 'q0b'),
    ('q1a', '0'): ('0', 'R', 'q1b'),
    ('q1a', '_'): ('_', 'N', 'q_rej'),

    ('q1b', '1'): ('X', 'R', 'q0'),
    ('q1b', '0'): ('0', 'R', 'q1'),
    ('q1b', '_'): ('_', 'N', 'q_rej'),
}


In [53]:
# Se finaliza em q2 vindo de q1, rejeita
final_rejecting = ['q1', 'q1a', 'q1b']
for state in final_rejecting:
    t_jump_paridade_1s[('q2', state)] = ('_', 'N', 'q_rej')

In [54]:
tape = '0_1_1_0'
tm = JumpingHeadTM(tape, t_jump_paridade_1s, 'q0', 'q_acc', 'q_rej')
print(tm.run())

(True, '0_X_X_0__', 5)


In [55]:
tm_err = JumpingHeadTM('1_0_0', t_jump_paridade_1s, 'q0', 'q_acc', 'q_rej')
print(tm_err.run())


(False, 'X_0_0__', 4)


Reconhecer cadeias com o padrão a_b_a_b... (intercalado)

In [77]:
t_jump_intercalado = {
    # Início: espera 'a'
    ('q0', 'a'): ('X', 'R', 'qa1'),
    ('q0', '_'): ('_', 'N', 'q_acc'),

    # Após 'a', espera 'b'
    ('qa1', 'b'): ('Y', 'R', 'qa2'),
    ('qa1', 'a'): ('a', 'N', 'q_rej'),
    ('qa1', '_'): ('_', 'N', 'q_rej'),
    ('qa1', 'X'): ('X', 'R', 'qa1'),
    ('qa1', 'Y'): ('Y', 'R', 'qa1'),

    # Após marcar b, transição para próximo ciclo
    ('qa2', 'a'): ('X', 'R', 'qa3'),
    ('qa2', '_'): ('_', 'N', 'q_acc'),
    ('qa2', 'b'): ('b', 'N', 'q_rej'),
    ('qa2', 'X'): ('X', 'R', 'qa2'),
    ('qa2', 'Y'): ('Y', 'R', 'qa2'),

    # qa3 espera b de novo
    ('qa3', 'b'): ('Y', 'R', 'qa2'),
    ('qa3', 'a'): ('a', 'N', 'q_rej'),
    ('qa3', '_'): ('_', 'N', 'q_acc'),
    ('qa3', 'X'): ('X', 'R', 'qa3'),
    ('qa3', 'Y'): ('Y', 'R', 'qa3'),
}


In [78]:
tm = JumpingHeadTM('a_b_a_b_', t_jump_intercalado, 'q0', 'q_acc', 'q_rej')
print(tm.run())


(True, 'X_Y_X_Y__', 5)


In [80]:
tm2 = JumpingHeadTM('a_b_b_', t_jump_intercalado, 'q0', 'q_acc', 'q_rej')
print(tm2.run())

(False, 'X_Y_b_', 3)
